## Is this just magic?  What is Numba doing to make code run quickly?

Let's define a trivial example function.

In [1]:
from numba import jit

In [2]:
@jit
def add(a, b):
    return a + b

In [3]:
add(1, 1)

2

Numba examines Python bytecode and then translates this into an 'intermediate representation'.  To view this IR, run (compile) `add` and you can access the `inspect_types` method.

In [4]:
add.inspect_types()

add (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-7a2ac56f16b6>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def add(a, b):

    # --- LINE 3 --- 
    #   a = arg(0, name=a)  :: int64
    #   b = arg(1, name=b)  :: int64
    #   $0.3 = a + b  :: int64
    #   del b
    #   del a
    #   $0.4 = cast(value=$0.3)  :: int64
    #   del $0.3
    #   return $0.4

    return a + b




Ok.  Numba is has correctly inferred the type of the arguments, defining things as `int64` and running smoothly.  

(What happens if you do `add(1., 1.)` and then `inspect_types`?)

In [4]:
add(1., 1.)

2.0

In [6]:
add.inspect_types()

add (int64, int64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-7a2ac56f16b6>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def add(a, b):

    # --- LINE 3 --- 
    #   a = arg(0, name=a)  :: int64
    #   b = arg(1, name=b)  :: int64
    #   $0.3 = a + b  :: int64
    #   del b
    #   del a
    #   $0.4 = cast(value=$0.3)  :: int64
    #   del $0.3
    #   return $0.4

    return a + b


add (float64, float64)
--------------------------------------------------------------------------------
# File: <ipython-input-2-7a2ac56f16b6>
# --- LINE 1 --- 
# label 0

@jit

# --- LINE 2 --- 

def add(a, b):

    # --- LINE 3 --- 
    #   a = arg(0, name=a)  :: float64
    #   b = arg(1, name=b)  :: float64
    #   $0.3 = a + b  :: float64
    #   del b
    #   del a
    #   $0.4 = cast(value=$0.3)  :: float64
    #   del $0.3
    #   return $0.4

    return a + b




### What about the actual LLVM code?

You can see the actual LLVM code generated by Numba using the `inspect_llvm()` method.  Since it's a `dict`, doing the following will be slightly more visually friendly.

In [12]:
for k, v in add.inspect_llvm().items():
    print(k, v)

(int64, int64) ; ModuleID = 'add'
source_filename = "<string>"
target datalayout = "e-m:e-i64:64-f80:128-n8:16:32:64-S128"
target triple = "x86_64-pc-windows-msvc"

@"_ZN08NumbaEnv8__main__7add$241Exx" = common local_unnamed_addr global i8* null
@.const.add = internal constant [4 x i8] c"add\00"
@PyExc_RuntimeError = external global i8
@".const.missing Environment" = internal constant [20 x i8] c"missing Environment\00"

; Function Attrs: norecurse nounwind
define i32 @"_ZN8__main__7add$241Exx"(i64* noalias nocapture %retptr, { i8*, i32 }** noalias nocapture readnone %excinfo, i64 %arg.a, i64 %arg.b) local_unnamed_addr #0 {
entry:
  %.14 = add nsw i64 %arg.b, %arg.a
  store i64 %.14, i64* %retptr, align 8
  ret i32 0
}

define i8* @"_ZN7cpython8__main__7add$241Exx"(i8* nocapture readnone %py_closure, i8* %py_args, i8* nocapture readnone %py_kws) local_unnamed_addr {
entry:
  %.5 = alloca i8*, align 8
  %.6 = alloca i8*, align 8
  %.7 = call i32 (i8*, i8*, i64, i64, ...) @PyArg_UnpackTu

## But there's a caveat

Now, watch what happens when we try to do something that is natural in Python, but not particularly mathematically sound:

In [7]:
def add_strings(a, b):
    return a + b

In [8]:
add_strings_jit = jit()(add_strings)

In [9]:
add_strings_jit('a', 'b')

'ab'

It worked, but what does `inspect_types` tell us?

In [10]:
add_strings_jit.inspect_types()

add_strings (str, str)
--------------------------------------------------------------------------------
# File: <ipython-input-7-d1008a9d4aa2>
# --- LINE 1 --- 
# label 0

def add_strings(a, b):

    # --- LINE 2 --- 
    #   a = arg(0, name=a)  :: pyobject
    #   b = arg(1, name=b)  :: pyobject
    #   $0.3 = a + b  :: pyobject
    #   del b
    #   del a
    #   $0.4 = cast(value=$0.3)  :: pyobject
    #   del $0.3
    #   return $0.4

    return a + b




## What's all this pyobject business?  

This means it has been compiled in `object` mode.  This can be a faster than regular python if it can do loop lifting, but not that fast.  
We want those `pyobjects` to be `int64` or another type that can be inferred by Numba. Your best bet is forcing `nopython` mode: this will throw an error if Numba finds itself in object mode, so that you _know_ that it can't give you speed.

For the full list of supported Python and NumPy features in `nopython` mode, see the Numba documentation here: http://numba.pydata.org/numba-doc/latest/reference/pysupported.html

## Figuring out what isn't working

In [11]:
%%file nopython_failure.py
from numba import jit

@jit
def add(a, b):
    for i in range(100):
        c = i
        f = i + 7
        l = c + f
        
    return a + b

add('a', 'b')

Writing nopython_failure.py


In [ ]:
!numba --annotate-html fail.html nopython_failure.py

[fail.html](fail.html)

## Forcing `nopython` mode

In [13]:
add_strings_jit = jit(nopython=True)(add_strings)

In [14]:
add_strings_jit('a', 'b')

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mInvalid use of + with parameters (str, str)
Known signatures:
 * (int64, int64) -> int64
 * (int64, uint64) -> int64
 * (uint64, int64) -> int64
 * (uint64, uint64) -> uint64
 * (float32, float32) -> float32
 * (float64, float64) -> float64
 * (complex64, complex64) -> complex64
 * (complex128, complex128) -> complex128
 * (uint8,) -> uint64
 * (uint16,) -> uint64
 * (uint32,) -> uint64
 * (uint64,) -> uint64
 * (int8,) -> int64
 * (int16,) -> int64
 * (int32,) -> int64
 * (int64,) -> int64
 * (float32,) -> float32
 * (float64,) -> float64
 * (complex64,) -> complex64
 * (complex128,) -> complex128
 * (bool,) -> int64
 * parameterized[0m
[0m[1m[1] During: typing of intrinsic-call at <ipython-input-7-d1008a9d4aa2> (2)[0m
[1m
File "<ipython-input-7-d1008a9d4aa2>", line 2:[0m
[1mdef add_strings(a, b):
[1m    return a + b
[0m    [1m^[0m[0m

This is not usually a problem with Numba itself but instead

In [15]:
from numba import njit

In [16]:
add_strings_jit = njit(add_strings)

In [17]:
add_strings_jit('a', 'b')

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mInvalid use of + with parameters (str, str)
Known signatures:
 * (int64, int64) -> int64
 * (int64, uint64) -> int64
 * (uint64, int64) -> int64
 * (uint64, uint64) -> uint64
 * (float32, float32) -> float32
 * (float64, float64) -> float64
 * (complex64, complex64) -> complex64
 * (complex128, complex128) -> complex128
 * (uint8,) -> uint64
 * (uint16,) -> uint64
 * (uint32,) -> uint64
 * (uint64,) -> uint64
 * (int8,) -> int64
 * (int16,) -> int64
 * (int32,) -> int64
 * (int64,) -> int64
 * (float32,) -> float32
 * (float64,) -> float64
 * (complex64,) -> complex64
 * (complex128,) -> complex128
 * (bool,) -> int64
 * parameterized[0m
[0m[1m[1] During: typing of intrinsic-call at <ipython-input-7-d1008a9d4aa2> (2)[0m
[1m
File "<ipython-input-7-d1008a9d4aa2>", line 2:[0m
[1mdef add_strings(a, b):
[1m    return a + b
[0m    [1m^[0m[0m

This is not usually a problem with Numba itself but instead

## Other compilation flags

There are two other main compilation flags for `@jit`

```python
cache=True
```

if you don't want to always want to get dinged by the compilation time for every run. This will actually save the compiled function into something like a `pyc` file in your `__pycache__` directory, so even between sessions you should have nice fast performance.

```python
nogil=True
```

This releases the GIL.  Note, however, that it doesn't do anything else, like make your program threadsafe.  You have to manage all of those things on your own (use `concurrent.futures`).